# After Original data
el procesamiento de data parece correcto y estable como esta.  
ahora seria cmabairle el modelo, en lugar de usar un ridge, usar otro.  
o uno de classification, ya que usa un regression.  
Debe tener la discusion de las metricas, y porque si o porque no

>> Python have templates reports

In [1]:
import pandas as pd
import re

In [2]:
df_raw=pd.read_csv('reviews.csv')
df = df_raw[['title','content','rating']]
df.head()

,title,content,rating
0,Pale imitation of better films,"Three words: ""Cool Hand Luke."" Same film, don...",1/10
1,Didactic and overlong,Another one of those overlong morally right-on...,1/10
2,Mediocre people reward mediocre cinema.,It bugs me that this movie is rated so high- n...,1/10
3,The plain & simple truth. It doesn\'t deserve ...,just read the title. Tough I think it's a pret...,1/10
4,Not the greatest movie of all time,Shawshank is nothing more than a fairy tale. ...,1/10


In [3]:
# print(df['rating'].unique())
df['rating'].unique()


array(['1/10', '2/10', '3/10', '4/10', '5/10', '6/10', '7/10', '8/10',
       '9/10', '10/10'], dtype=object)

In [4]:

df = df[~df['rating'].isna()]
df['rating']=[int(x.split('/')[0]) for x in df['rating']]
df

,title,content,rating
0,Pale imitation of better films,"Three words: ""Cool Hand Luke."" Same film, don...",1
1,Didactic and overlong,Another one of those overlong morally right-on...,1
2,Mediocre people reward mediocre cinema.,It bugs me that this movie is rated so high- n...,1
3,The plain & simple truth. It doesn\'t deserve ...,just read the title. Tough I think it's a pret...,1
4,Not the greatest movie of all time,Shawshank is nothing more than a fairy tale. ...,1
...,...,...,...
20306,extremely scary,peter Lorre is the best child serial killer ci...,10
20307,Classic Mystery,I thoroughly enjoyed watching this film. Seein...,10
20308,"One of the most influential films ever, almost...","One of the most influential films ever, almost...",10
20309,M a few thoughts,"I will not take time to praise the film, as ot...",10


In [5]:
df.isna().sum()


title      0
content    0
rating     0
dtype: int64

In [6]:
df['rating'].value_counts()

9     2496
8     2493
10    2492
7     2432
6     2202
5     1925
1     1790
4     1612
3     1489
2     1380
Name: rating, dtype: int64

# Preprocess Text

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stopword = stopwords.words('english')

def clean_text(text):
    text = text.replace('\n','. ')#.replace('','').replace('','').replace('','')
    return text

def remove_punct(text, lower=False):
    if(type(text)==str):
        text = text.replace('\n','. ')#.replace('','').replace('','').replace('','')
        text = text.replace(',',' ').replace('!','').replace('?',' ')
        text = text.replace('.',' ').replace('#','').replace('$',' ')
        text = text.replace('^',' ').replace('&','and').replace(';',' ')
        text = text.replace('  ',' ')
        return text
    elif(type(text)==list):
        _words=[]
        for w in text:
            if w.isalnum():
                _words.append(w if not lower else w.lower())
#         print('remove_punct',len(_words))
        return _words

def get_sentences(text):
    return sent_tokenize(text)

def get_tokens(text):
    words =[]
    for w in word_tokenize(text):
        if w.isalnum():
            words.append(w)
    return words
    

def remove_stopwords(words):
    just_words=[]
    for word in words:
        if word.lower() not in stopword:
            just_words.append(word)
#     print('just_words',len(just_words))
    return just_words
    
    
def lemmatize(text):
    record_lemmatized = [lemmatizer.lemmatize(token) for token in text]
    return record_lemmatized
    
    
def list_to_string(ls):
    return " ".join(ls)
    
# clean words: boooook
# Remove non stop words of 2 letters

In [28]:
#test


In [27]:
df['content'][0]

'Three words: "Cool Hand Luke."  Same film, done better, done earlier.  For that matter, is this film any better than other Steven King "novelettes" such as "Stand By Me"? All in all, it probably ranks a 6 or a 7, but since people on this site have lost their minds as regards this film, I give it a 1 in one man\'s attempt at sanity.'

In [42]:
# Cleaning approach 1:
    # tokenize everything, remove stop words, lower all, remove punctutation

from nltk.stem import PorterStemmer, WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# set tokens each description
dataset_tokens= [ get_tokens(x) for x in df['content']] 

# remove stopwords
dataset_nostopwords= [remove_stopwords(x) for x in dataset_tokens]

# remove punctuation
dataset_main_words= [remove_punct(x, lower=False) for x in dataset_nostopwords ]

# Lower all
# dataset_main_words_lower=[x.lower() for x in dataset_main_words]

# Lemmatize words :  IT requires POS
# record_lemmatized = [lemmatizer.lemmatize(token) for token in dataset_main_words]


# Back to string
dataset_clean_text = [ list_to_string(x) for x in dataset_main_words]

def preprocess_text_ap1(x):
       # tokenize everything, remove stop words, lower all, remove punctutation
    # set tokens each description
    dataset_tokens= get_tokens(x)
    # remove stopwords
    dataset_nostopwords= remove_stopwords(dataset_tokens)
    # remove punctuation
    dataset_main_words= remove_punct(dataset_nostopwords, lower=False)
    # Lower all
    # dataset_main_words_lower=[x.lower() for x in dataset_main_words]
    # Back to string
    dataset_clean_text =  list_to_string(dataset_main_words)
    return dataset_clean_text

# METHOD 1 TFIDF

compare to a manual method  
separate the test-train in equal samples

The training was giving a lot of 8+ ratings. I rerun this with stratify


# Original data


In [43]:
original=False

In [79]:
# v2, improved gpt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Assuming you have a list of movie reviews and their corresponding ratings
if original:
    reviews = df['content']
    ratings = df['rating']
else:
    reviews = dataset_clean_text
    ratings = df['rating']
    

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.3, random_state=42, stratify=ratings)
# 
# Create a TF-IDF vectorizer with preprocessing options
vectorizer = TfidfVectorizer(
    lowercase=True,
    strip_accents='unicode',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=5000 # features made by term frequency
)

# Fit the vectorizer on the training reviews and transform the data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)




############# RIDGE  !! USe different model

# Create a Ridge regression model with regularization
regressor = Ridge(alpha=0.5)

# Train the Ridge regression model
regressor.fit(X_train_tfidf, y_train)

# Predict ratings for the testing reviews
y_pred = regressor.predict(X_test_tfidf)

# Evaluate the model using mean squared error (MSE) and mean absolute error (MAE)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Perform cross-validation to get a more robust performance estimate
cv_scores = cross_val_score(regressor, X_train_tfidf, y_train, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()
print("Cross-Validated MSE:", cv_mse)

"""Pre-Processed:
Mean Squared Error: 4.444501189186152
Mean Absolute Error: 1.6877876810286814
Cross-Validated MSE: 4.424840704022955"""

Mean Squared Error: 4.582455908824896
Mean Absolute Error: 1.707997277161021
Cross-Validated MSE: 4.431198160575902


'Pre-Processed:\nMean Squared Error: 4.444501189186152\nMean Absolute Error: 1.6877876810286814\nCross-Validated MSE: 4.424840704022955'

In [80]:
review_test="Imagine what you'd do, when you wake from a bad dream, to find you're held by four square walls, for as long as someone deems. No idea why you're trapped, what you've done, why you're kidnapped, just a ceaseless line of dumplings going down your gyoza hatch. Now some fifteen years have passed, every question has been asked, and you're suddenly set free, can start your own avenging spree. Before you do you need to feed, by eating something that's in need, so an octopus is ordered, and head first you cross the borders. But things aren't what they might seem, tied and tethered and undreamed, as the puppet master hovers, manipulates what you'll discover. Some films you cannot watch too often and this is one of the greatest pieces of cinematic brilliance ever created."
rating_test=10
X_test_tfidf = vectorizer.transform([review_test])
predicted_rating = regressor.predict(X_test_tfidf)
print(f"Predicted: {'{:.2f}'.format(predicted_rating[0])} of {rating_test}")
# Sin procesar Predicted: 9.44 of 10
# PRocesado: Predicted: 9.47 of 10
#modded 8.5
#mod2: Predicted: 9.25 of 10


Predicted: 9.77 of 10


In [81]:
review_test="Was this a missed warning sign? The current climate in America is kind of portayed here. Though some would argue, that it doesn't depict all of America, not even just the one side who seems to have a glutton for punishment and seems to like to vote against their own interests. So while there is some relevance to current events, this go far out.Still if you feel uneasy watching this, it doesn't mean something is wrong with you. Quite the opposite is the case, everything is right with you. Quite ridiculous at times, it is there for entertainment purposes ... no really! They weren't trying to do a documentary! All kidding aside, this can be viewed as fun - no matter what your political background is."
rating_test=7
X_test_tfidf = vectorizer.transform([review_test])
predicted_rating = regressor.predict(X_test_tfidf)

print(f"Predicted: {'{:.2f}'.format(predicted_rating[0])} of {rating_test}")

# SIn procesar Predicted: 2.89 of 7
# PRocesar Predicted: 2.79 of 7
# mod1: Predicted: 4.65 of 7
#mod2: Predicted: 3.80 of 7


Predicted: 3.42 of 7


In [82]:
import math

In [83]:
_reviews=[
    "Eugenics is the study and practice of selective breeding applied to humans, with the aim of improving the species. In a historical and broader sense, eugenics can also be a study of \"improving human genetic qualities. \"Advocates of eugenics sought to counter what they regarded as spoiler of genetics dynamics within the human gene pool, specifically in regard to congenital disorders and factors relating to the heritability of IQ. This movie is a clear eugenics message. That's to say Eugencists are Striking Back. And Eugenics is totally unethical, and leads to racism.",
    """I find it a bit ironic that a movie about the dumbing down of America sums up the message in the first few minutes and then spends nearly 90 minutes repeating the same jokes to get the point across. The message is a bit frighteningly on point in a time when, what seem to be, complete idiots on social media make millions. And I'm guessing many of us have known families like both that were portrayed at the start. I think those two things gain this movie a lot of its accolades despite the 'story'. It had its moments here and there, but as a whole it wasn't as great as the hype.""",
    """The story of the movie is just perfect. All futuristic movies describe an advanced civilization. This movie chooses the other way around, outsmarting all other super-advanced-Hollywood-fictional-hi-tech scripts. May be movie was supposed to be rated with "R+"; the emphasis on sex-related "humor" is off the charts. In my opinion, the prime target for the movie is below 14 years of age, like may be 12. It definitely doesn't address for 25+ people. It's not funny at all, I don't remember I have laughed even once. Story is short; he solves exactly one problem. I think this story could be expressed in far better ways than this movie. It's a brilliant idea, but bad screenplay. Movie tried to tell "how stupid can we get" in a stupid (or, may be, "poor") way. From this point of view, it's successful; it's all about stupid stuff, and it's all told in stupid way. I, however, still believe that it could be a movie that still shows "how stupid can we get" in a smarter way. As a conclusion; it's recommended to be watched but don't expect real comedy or smart jokes. Don't try to watch during your romantic moments, not to mention keep this movie away from your children.""",
    """I hate indian movies they're garbage just terrible I mean seriously all these singing, dancing and all the dramatic crying they all look the same to me BUT this is the only movie which I liked original screenplay absolutely flawless movie by a genius director I still can't believe this is indian movie highly recommended for horror genre fans""",
    """Tumbbad is not a common movie. It will leave you gripped to your seat making you think "what would happen next". Whole movie is so beautifully shot no wonder it is shot in natural light and the beauty is visible. This movie is a feast for your eyes with cinematic excellence is on full display. You wouldnt even have time to wander off your mind but watch it with eyes wide open. You won't enjoy it on small screen but have to watch in cinema halls. Mind you this movie took 6 years in making can you believe in the perseverance of the people who clung onto movie for 6 years despite harships.""",
    """Tumbbad is IMO the best Indian horror movie ever, with a strong script and incredibly good shadowy visuals and great sound design (although I will cut a point for the loud RGV-esque background score). Focused direction and a solid lead performance from Sohum Shah (who also produced the film)."""
]
_ratings=[
    1,
    4,
    1,
    8,
    10,
    9,
]
for review_test,rating_test in zip(_reviews,_ratings):
    X_test_tfidf = vectorizer.transform([review_test])
    predicted_rating = regressor.predict(X_test_tfidf)
#     print(f"Predicted: {'{:.2f}'.format(predicted_rating[0])} of {rating_test}")
    print(f"Predicted: {math.floor(predicted_rating[0])} of {rating_test}")
    
    
"""Just content
Predicted: 4 of 1
Predicted: 6 of 4
Predicted: 4 of 1
Predicted: 6 of 8
Predicted: 9 of 10
Predicted: 7 of 9

v2
Predicted: 3 of 1
Predicted: 6 of 4
Predicted: 4 of 1
Predicted: 5 of 8
Predicted: 8 of 10
Predicted: 6 of 9

Preprocessed:
Predicted: 2 of 1
Predicted: 6 of 4
Predicted: 4 of 1
Predicted: 6 of 8
Predicted: 8 of 10
Predicted: 6 of 9

#modified TDIFtokenize
Predicted: 5 of 1
Predicted: 5 of 4
Predicted: 3 of 1
Predicted: 4 of 8
Predicted: 7 of 10
Predicted: 6 of 9

# MOD 2
Predicted: 3 of 1
Predicted: 6 of 4
Predicted: 4 of 1
Predicted: 4 of 8
Predicted: 9 of 10
Predicted: 6 of 9
"""

Predicted: 3 of 1
Predicted: 6 of 4
Predicted: 4 of 1
Predicted: 6 of 8
Predicted: 8 of 10
Predicted: 7 of 9


'Just content\nPredicted: 4 of 1\nPredicted: 6 of 4\nPredicted: 4 of 1\nPredicted: 6 of 8\nPredicted: 9 of 10\nPredicted: 7 of 9\n\nv2\nPredicted: 3 of 1\nPredicted: 6 of 4\nPredicted: 4 of 1\nPredicted: 5 of 8\nPredicted: 8 of 10\nPredicted: 6 of 9\n\nPreprocessed:\nPredicted: 2 of 1\nPredicted: 6 of 4\nPredicted: 4 of 1\nPredicted: 6 of 8\nPredicted: 8 of 10\nPredicted: 6 of 9\n\n#modified TDIFtokenize\nPredicted: 5 of 1\nPredicted: 5 of 4\nPredicted: 3 of 1\nPredicted: 4 of 8\nPredicted: 7 of 10\nPredicted: 6 of 9\n\n# MOD 2\nPredicted: 3 of 1\nPredicted: 6 of 4\nPredicted: 4 of 1\nPredicted: 4 of 8\nPredicted: 9 of 10\nPredicted: 6 of 9\n'

In [77]:
# PERSONAL Reviews
_reviews=[
"This is one of my most favorite movies, I have enjoyed every aspect of it, from camera, script, effects, and music. All is perfect",
    "This was a waste of time; Everything was not good, music, dialogue, characters, screeplany. I wish I have spent my time better",
    "I am sad I haven't knew about this movie before. It has everything on it, not just for a nische, but for everyone."
]
_ratings=[
    9,
    1,
    10,
]
for review_test,rating_test in zip(_reviews,_ratings):
    X_test_tfidf = vectorizer.transform([review_test])
    predicted_rating = regressor.predict(X_test_tfidf)
#     print(f"Predicted: {'{:.2f}'.format(predicted_rating[0])} of {rating_test}")
    print(f"Predicted: {math.floor(predicted_rating[0])} of {rating_test}")

"""Sin procesar:
Predicted: 9 of 9
Predicted: 2 of 1
Predicted: 6 of 10

Procesando:
Predicted: 9 of 9
Predicted: 2 of 1
Predicted: 6 of 10

ModTTDIF
Predicted: 9 of 9
Predicted: 1 of 1
Predicted: 6 of 10
"""

Predicted: 9 of 9
Predicted: 1 of 1
Predicted: 6 of 10


'Sin procesar:\nPredicted: 9 of 9\nPredicted: 2 of 1\nPredicted: 6 of 10\n\nProcesando:\nPredicted: 9 of 9\nPredicted: 2 of 1\nPredicted: 6 of 10\n\nModTTDIF\nPredicted: 9 of 9\nPredicted: 1 of 1\nPredicted: 6 of 10\n'

In [78]:
# More reviews
_reviews=["""This show has a beginning and an actual ending. Not every show can offer that. This TV show is underrated and that's a shame. People need to watch this!""",
         """Replicating the 90s movie was tough, I acknowledge that.
However, one thing stroke me and I won't be watching any more episodes (currently at s1e10): They find Pieters (the virus' creator) in the container, and there is no mention-not even a thought- about asking him about the cure or having him work on a vaccine.
She was point-blank ready to shoot him, I mean, that was the point where it became obvious that the whole series had been designed to draaaag and draaaag and draaaag. Absolutely no credibility.
What I would expect from a 12 monkeys tv series after so many years, would be to have the opportunity to dive into the whole 12 monkeys dystopian universe, learn more about the characters and a bit more details about the twists in the original motion picture.
Really loving the effort from the cast. From a director's stand-point, its not awesome but its not bad. But in terms of script, the whole thing is too messed up, overly complicated, overly paced and really why they didn't ask Pieters the cure or have him work on the vaccine?
Seriously, I wish the creators of Dark had gotten their hands on the script and the realization as a whole.""",
         
         ]
_ratings=[9,
         3]

for review_test,rating_test in zip(_reviews,_ratings):
    X_test_tfidf = vectorizer.transform([review_test])
    predicted_rating = regressor.predict(X_test_tfidf)
    print(f"Predicted: {math.floor(predicted_rating[0])} of {rating_test}")

"""
PreProcesado:
Predicted: 4 of 9
Predicted: 3 of 3"""

Predicted: 4 of 9
Predicted: 3 of 3


'\nPreProcesado:\nPredicted: 4 of 9\nPredicted: 3 of 3'

# METHOD 2 
# Embedding: word2vec  + Neural Network

In [51]:
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Flatten
# from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [63]:
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# for w in text:
#             if w.isalnum():
#                 _words.append(w if not lower else w.lower())


# for r in df['rating']

# df['content_ed']=[  (z for z in x if z.isalnum())   for x in [word_tokenize(y) for y in df['content']] ]

contents=[]
for c in df['content']:
    tokens=[]
    for t in word_tokenize(c):
        if t not in stop_words and t.isalnum():
            tokens.append(t)
    contents.append(tokens)
df['content_ed']=contents


In [64]:
# Assuming you have a list of movie reviews and their corresponding ratings
reviews = df['content']
ratings = df['rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.2, random_state=42)

# Tokenize and train Word2Vec on the training reviews
tokenized_reviews = [review.split() for review in X_train]
word2vec_model = Word2Vec(tokenized_reviews, vector_size=100, window=5, min_count=1)

# Convert each review into a sequence of word embeddings
X_train_sequences = [[word2vec_model.wv[word] for word in review.split()] for review in X_train]
X_test_sequences = [[word2vec_model.wv[word] for word in review.split()] for review in X_test] #<Error

# Pad the sequences to a fixed length
max_sequence_length = max(len(sequence) for sequence in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Create a neural network model
model = Sequential()
model.add(Flatten(input_shape=(max_sequence_length, word2vec_model.vector_size)))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, verbose=1)

# Predict ratings for the testing reviews
y_pred = model.predict(X_test_padded).flatten()

# Evaluate the model using mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

KeyError: "Key 'meritocracy,' not present"

In [69]:
index=0
for i,t in enumerate(df['content_ed']):
    if('meritocracy' in t):
        print('found on ', t)
        index=i
print(index)

        

found on  ['In', 'meritocracy', 'success', 'fortune', 'reserved', 'deserve', 'develop', 'solid', 'plans', 'according', 'talents', 'abilities', 'execute', 'plans', 'hard', 'work', 'determination', 'Anyone', 'rise', 'top', 'lucky', 'Cinderella', 'plucked', 'cinders', 'gussied', 'gowns', 'meritocracy', 'represents', 'heights', 'perfect', 'egalitarian', 'society', 'I', 'started', 'nothing', 'ended', 'everything', 'I', 'ever', 'desired', 'achieve', 'dreams', 'try', 'The', 'promise', 'unobstructed', 'sunshine', 'top', 'mountain', 'becomes', 'justification', 'bitter', 'competition', 'backstabbing', 'deceit', 'callousness', 'You', 'climb', 'crooked', 'ladder', 'make', 'straight', 'one', 'perhaps', 'last', 'feel', 'secure', 'afford', 'kind', 'confident', 'generous', 'It', 'easy', 'nice', 'rich', 'mother', 'film', 'Jang', 'one', 'point', 'long', 'crooked', 'ladder', 'sometimes', 'rungs', 'give', 'beneath', 'grip', 'sometimes', 'dangerously', 'greased', 'climbed', 'sometimes', 'ladder', 'simply',

title                           Meritocracy: it\'s metaphorical
content       In a meritocracy, success and fortune are rese...
rating                                                       10
content_ed    [In, meritocracy, success, fortune, reserved, ...
Name: 852, dtype: object

In [70]:
df.iloc[index]

title                           Meritocracy: it\'s metaphorical
content       In a meritocracy, success and fortune are rese...
rating                                                       10
content_ed    [In, meritocracy, success, fortune, reserved, ...
Name: 852, dtype: object

# Method 3
# Transformers

In [71]:
from transformers import BertTokenizer, BertModel
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming you have a list of movie reviews and their corresponding ratings
reviews = df['content']
ratings = df['rating']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.2, random_state=42)

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the reviews
X_train_encoded = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt') #ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
X_test_encoded = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt')

# Obtain the BERT embeddings for the reviews
with torch.no_grad():
    X_train_embeddings = model(**X_train_encoded).last_hidden_state.mean(dim=1)
    X_test_embeddings = model(**X_test_encoded).last_hidden_state.mean(dim=1)

# Create a simple feed-forward neural network
class RatingPredictor(nn.Module):
    def __init__(self, input_size):
        super(RatingPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert data to tensors and move to device
X_train_tensors = torch.tensor(X_train_embeddings).to(device)
y_train_tensors = torch.tensor(y_train).unsqueeze(1).to(device)
X_test_tensors = torch.tensor(X_test_embeddings).to(device)

# Initialize the model and move to device
model = RatingPredictor(X_train_tensors.size(1)).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
batch_size = 32
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(0, X_train_tensors.size(0), batch_size):
        inputs = X_train_tensors[i:i+batch_size]
        labels = y_train_tensors[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / X_train_tensors.size(0)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensors).squeeze().cpu().numpy()

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


/Users/ivan/opt/anaconda3/envs/sem2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|████████████████████████| 232k/232k [00:00<00:00, 5.58MB/s]
Downloading: 100%|████████████████████████| 28.0/28.0 [00:00<00:00, 9.90kB/s]
Downloading: 100%|███████████████████████████| 570/570 [00:00<00:00, 182kB/s]
Downloading: 100%|████████████████████████| 440M/440M [00:16<00:00, 26.0MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you a

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).